In [1]:
import pandas as pd
import numpy as np

In [2]:
import include.GA_clsData as DA
import include.GA_clsModel as MO

### Genes and params

In [3]:
df = pd.read_csv('hyperparameters/AdaBoostClassifier_hyperparameters.csv')

In [4]:
df.sort_values(by='fitness', ascending=False, inplace=True)

In [5]:
genes={}
for _, row in df.head(1).iterrows():
    for col in df.columns:
        if col == 'fitness': continue
        value = 0
        if col[0] == 'i': value = int(row[col])
        if col[0] == 'f': value = float(row[col])
        genes[col] = value

In [6]:
params = []
params.append(['periods_away_to_predict',1])
params.append(['get_target',True])
params.append(['start_new', True])
params.append(['initial_population', 1])
#params.append(['training_file', 'data/EURUSD_M5_2014-01-01--2014-01-31.csv'])
params.append(['training_file', 'data/EURUSD_M5_2014-01-01--2018-12-31.csv'])

param_dict = {}
for p in params:
    param_dict[p[0]] = p[1]
params = param_dict

### Data

In [7]:
data = DA.Data(genes,params)

In [8]:
data.getDataSet()

In [9]:
data.getXy(number_validations = 10000)

In [10]:
X_train = data.X_train
X_val = data.X_val

y_train = data.y_train
y_val = data.y_val

### Classifiers

In [11]:
import include.Classifiers as cl
classifiers = cl.getClassifiers()

### Loop through and train

In [12]:
totry = {'SVC'}

In [ ]:
df = pd.DataFrame()
target = y_val.reset_index(drop=True)
target = [1 if p==True else 0 for p in target]
df['target'] = target
for classifier in classifiers:
    classifier_name = type(classifier).__name__
    
    #if classifier_name not in totry: continue
    
    params['classifier_name'] = classifier_name

    model = MO.Model(genes, params, data)
    accuracy = model.train()
    print(f'Accuracy: {accuracy}')
    
    
    try:
        #pred = model.classifier.predict_proba(X_val)
        #pred = pred[:,0]
        pred = model.classifier.predict(X_val)
        pred = [1 if p==True else 0 for p in pred]
        df[classifier_name] = pred
    except:
        pass

Training: AdaBoostClassifier
Accuracy: 53.5
Training: BaggingClassifier
Accuracy: 52.0
Training: BernoulliNB
Accuracy: 52.7
Training: CalibratedClassifierCV
Accuracy: 53.5
Training: DecisionTreeClassifier
Accuracy: 51.2
Training: ExtraTreeClassifier
Accuracy: 51.0
Training: ExtraTreesClassifier
Accuracy: 54.1
Training: GradientBoostingClassifier


In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df_true = df[df['target']==1]
df_false = df[df['target']==0]

ix = min(df_true.shape[0], df_false.shape[0])
df_true = df_true.head(ix)
df_false = df_false.head(ix)

df = pd.concat([df_true, df_false], axis=0)
df = df.sample(frac = 1)

df.reset_index(drop=True,inplace=True)

In [ ]:
df.shape

In [ ]:
df.to_csv('model_preds.csv', index=False)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [ ]:
model = xgb.XGBClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_val)

In [ ]:
acc = np.float(f'{accuracy_score(y_val, pred) * 100:0.1f}')
print(acc)